In [9]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import OCC
import pandas as pd
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.TopoDS import TopoDS_Shape, TopoDS_Compound, TopoDS_Iterator
from OCC.Core.TopAbs import TopAbs_ShapeEnum, TopAbs_SOLID
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.gp import gp_Pnt
from OCC.Core import BRepBndLib
from IPython.display import HTML, display

def get_bounding_box(shape):
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    min_pnt = bbox.CornerMin()
    max_pnt = bbox.CornerMax()
    return (min_pnt.X(), min_pnt.Y(), min_pnt.Z(), max_pnt.X(), max_pnt.Y(), max_pnt.Z())
    
def find_all_solids(shape):
    solids = []
    if shape.ShapeType() == TopAbs_SOLID:
        solids.append(shape)
    elif shape.ShapeType() == TopAbs_ShapeEnum.TopAbs_COMPOUND:
        iterator = TopoDS_Iterator(shape)
        while iterator.More():
            sub_shape = iterator.Value()
            solids.extend(find_all_solids(sub_shape))
            iterator.Next()
    return solids

step_file_path = "C25001-1-0101.step"

step_reader = STEPControl_Reader()
status = step_reader.ReadFile(step_file_path)

if status == 1:
    step_reader.TransferRoots()
    compound_shape = step_reader.Shape(1)
    all_solid_bodies = find_all_solids(compound_shape)
    print(f"Found a total of {len(all_solid_bodies)} solid bodies.")

    body_properties = []
    for i, body in enumerate(all_solid_bodies):
        props = GProp_GProps()
        brepgprop.VolumeProperties(body, props)
        volume = props.Mass()
        com = props.CentreOfMass()
        bbox = get_bounding_box(body)
        body_properties.append({'shape': body, 'volume': volume, 'com': com, 'bbox': bbox})
        if (i + 1) % 50 == 0:
            print(f"Processed {i + 1}/{len(all_solid_bodies)} bodies...")

    print("\nVolume, center of mass, and bounding box calculated for all bodies.")

    unique_shapes = {}  # Dictionary to store unique property tuples and their counts
    volume_tolerance = 1e-6
    com_tolerance = 1e-6
    bbox_tolerance = 1e-6

    for prop1 in body_properties:
        volume = prop1['volume']
        com = prop1['com']
        bbox = prop1['bbox']

        # Create a hashable tuple representing the properties
        property_tuple = (
            f"{volume:.8f}",
            f"{com.X():.8f}_{com.Y():.8f}_{com.Z():.8f}",
            tuple(f"{b:.8f}" for b in bbox)
        )

        is_unique = True
        for unique_tuple, data in unique_shapes.items():
            vol2_str, com2_str, bbox2_tuple_str = unique_tuple
            vol2 = float(vol2_str)
            com2_coords = [float(c) for c in com2_str.split('_')]
            com2 = gp_Pnt(*com2_coords)
            bbox2 = tuple(float(b) for b in bbox2_tuple_str)

            if abs(volume - vol2) < volume_tolerance and \
               com.Distance(com2) < com_tolerance and \
               all(abs(b1 - b2) < bbox_tolerance for b1, b2 in zip(bbox, bbox2)):
                unique_shapes[unique_tuple]['count'] += 1
                is_unique = False
                break

        if is_unique:
            unique_shapes[property_tuple] = {'count': 1, 'shape': prop1['shape']}

    print("\nIdentified unique bodies and their quantities (with bounding box check):")
    
    bill_of_materials_data = []
    unique_shapes_list = list(unique_shapes.items())  # Convert to a list for indexing
    for i, (unique_prop, data) in enumerate(unique_shapes_list):
        identifier = f"Part_{i + 1}"
        quantity = data['count']
        volume = unique_prop[0]
        bbox = unique_prop[2]
        bbox_dims = (
            f"{float(bbox[3]) - float(bbox[0]):.3f}",
            f"{float(bbox[4]) - float(bbox[1]):.3f}",
            f"{float(bbox[5]) - float(bbox[2]):.3f}",
        )
        bill_of_materials_data.append({
            'Part Identifier': identifier,
            'Quantity': quantity,
            'Volume': volume,
            'BBox X': bbox_dims[0],
            'BBox Y': bbox_dims[1],
            'BBox Z': bbox_dims[2],
        })

    bill_of_materials_df = pd.DataFrame(bill_of_materials_data)

    # Convert DataFrame to HTML with styling for full width
    html_table = bill_of_materials_df.to_html(index=False)
    styled_html = f"""
    <style>
        .dataframe {{
            width: 100% !important;
        }}
    </style>
    {html_table}
    """
    display(HTML(styled_html))

    print(f"\nTotal number of unique parts: {len(bill_of_materials_df)}")

else:
    print(f"Error reading STEP file: {step_file_path}")

Found a total of 2140 solid bodies.
Processed 50/2140 bodies...
Processed 100/2140 bodies...
Processed 150/2140 bodies...
Processed 200/2140 bodies...
Processed 250/2140 bodies...
Processed 300/2140 bodies...
Processed 350/2140 bodies...
Processed 400/2140 bodies...
Processed 450/2140 bodies...
Processed 500/2140 bodies...
Processed 550/2140 bodies...
Processed 600/2140 bodies...
Processed 650/2140 bodies...
Processed 700/2140 bodies...
Processed 750/2140 bodies...
Processed 800/2140 bodies...
Processed 850/2140 bodies...
Processed 900/2140 bodies...
Processed 950/2140 bodies...
Processed 1000/2140 bodies...
Processed 1050/2140 bodies...
Processed 1100/2140 bodies...
Processed 1150/2140 bodies...
Processed 1200/2140 bodies...
Processed 1250/2140 bodies...
Processed 1300/2140 bodies...
Processed 1350/2140 bodies...
Processed 1400/2140 bodies...
Processed 1450/2140 bodies...
Processed 1500/2140 bodies...
Processed 1550/2140 bodies...
Processed 1600/2140 bodies...
Processed 1650/2140 bodi

Part Identifier,Quantity,Volume,BBox X,BBox Y,BBox Z
Part_1,1,1208280.18592949,336.125,450.000,12.000
Part_2,1,438681.24124930,152.200,150.000,152.400
Part_3,1,14327617.70526127,600.000,40.000,600.000
Part_4,1,791089.99999983,145.000,20.000,277.100
Part_5,1,791089.99999975,145.000,20.000,277.100
Part_6,1,346345.13322354,200.000,8.000,220.000
Part_7,1,584745.13322354,369.000,8.000,200.000
Part_8,1,584745.13322354,369.000,8.000,200.000
Part_9,1,584745.13322354,369.000,8.000,200.000
Part_10,1,584745.13322354,369.000,8.000,200.000



Total number of unique parts: 2076
